# Packages

In [9]:
import pandas as pd
import numpy as np

# Importation des données

In [38]:
df_stationary = pd.read_csv("df_stationary.csv", index_col="date")
df_stationary_test = pd.read_csv("df_stationary_test.csv", index_col="date")

In [39]:
df_stationary.columns

Index(['UNRATE', 'TB3MS', 'RPI', 'INDPRO', 'DPCERA3M086SBEA', 'S&P 500',
       'BUSLOANS', 'CPIAUCSL', 'OILPRICEx', 'M2SL', 'USREC'],
      dtype='object')

In [41]:
df_stationary_test.columns

Index(['UNRATE', 'TB3MS', 'RPI', 'INDPRO', 'DPCERA3M086SBEA', 'S&P 500',
       'BUSLOANS', 'CPIAUCSL', 'OILPRICEx', 'M2SL', 'USREC'],
      dtype='object')

# RIDGE

In [35]:
# ==========================================================
# 🔹 Ridge Regression + Bagging (pseudo-OOS)
#    → même structure que LinearRegression
# ==========================================================
import numpy as np
import pandas as pd
import pickle
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ---------- Paramètres généraux ----------
h = 12
min_train_n = 36
winsor_level = 0.01
norm_var = True
target_col = "UNRATE"

# Fenêtres d’évaluation / test
eval_start = pd.Timestamp("1983-01-01")
eval_end   = pd.Timestamp("1989-12-31")
test_start = pd.Timestamp("1990-01-01")
test_end   = pd.Timestamp("2025-12-31")

# Bagging (moving-block bootstrap)
use_bagging = True
B_boot = 30
L_block = 12
rng = np.random.default_rng(123)

# Grille de recherche pour alpha
alpha_grid = np.logspace(-4, 4, 30)

# ---------- Fichiers ----------
RIDGE_PKL  = "ridge_regression.pkl"
RIDGE_META = "ridge_regression_meta.csv"

# ---------- Préparation des données ----------
def _ensure_ms_index(df):
    if "date" in df.columns:
        df = df.set_index("date")
    idx = pd.to_datetime(df.index)
    df.index = idx.to_period("M").to_timestamp(how="start")
    return df.asfreq("MS")

df_all = _ensure_ms_index(df_stationary).sort_index()
if target_col not in df_all.columns:
    raise ValueError(f"La colonne cible '{target_col}' est absente de df_stationary.")

y_all = df_all[target_col].astype(float)
X_all = df_all.drop(columns=[target_col]).astype(float)
features = list(X_all.columns)

print(f"✅ Données prêtes : {df_all.index.min().date()} → {df_all.index.max().date()} | n={len(df_all)} | freq=MS")
print(f"Features ({len(features)}): {features[:6]}{' ...' if len(features)>6 else ''}")

# ---------- Préproc ----------
def fit_preproc(X, wins=0.01, do_norm=True):
    lower = X.quantile(wins)
    upper = X.quantile(1 - wins)
    Xw = X.clip(lower=lower, upper=upper, axis=1)
    if do_norm:
        mean = Xw.mean()
        std = Xw.std().replace(0, 1)
        Xn = (Xw - mean) / std
        prep = {"lower": lower, "upper": upper, "mean": mean, "std": std, "norm": True}
        return Xn, prep
    else:
        prep = {"lower": lower, "upper": upper, "mean": None, "std": None, "norm": False}
        return Xw, prep

def apply_preproc(X, prep):
    Xp = X.clip(lower=prep["lower"], upper=prep["upper"], axis=1)
    if prep["norm"]:
        Xp = (Xp - prep["mean"]) / prep["std"].replace(0, 1)
    return Xp

# ---------- Bootstrap utils ----------
def block_bootstrap_rows(index, L, rng):
    n = len(index)
    if n < 3:
        return np.arange(n)
    L = max(2, min(int(L), n - 1))
    nb = int(np.ceil(n / L))
    starts = rng.integers(0, n - L + 1, size=nb)
    ix = np.concatenate([np.arange(s, s + L) for s in starts])[:n]
    return ix

def bagged_predict_ridge(X_tr_raw, y_tr, x_fore_raw, prep, alpha, B, L, rng):
    X_tr_p = apply_preproc(X_tr_raw, prep)
    base = Ridge(alpha=alpha, fit_intercept=True)
    base.fit(X_tr_p, y_tr.values)
    yhat_base = float(base.predict(apply_preproc(x_fore_raw, prep))[0])

    preds = []
    for _ in range(B):
        ix = block_bootstrap_rows(X_tr_raw.index, L, rng)
        Xb = X_tr_raw.iloc[ix]
        yb = y_tr.iloc[ix]
        Xb_p = apply_preproc(Xb, prep)
        m = Ridge(alpha=alpha, fit_intercept=True)
        m.fit(Xb_p, yb.values)
        preds.append(float(m.predict(apply_preproc(x_fore_raw, prep))[0]))
    return float(np.mean(preds)), np.array(preds), yhat_base

# ---------- Tuning ----------
def tune_ridge(X, y):
    """Recherche du meilleur alpha via GridSearchCV (MAE)."""
    model = Ridge(fit_intercept=True)
    grid = GridSearchCV(model, {"alpha": alpha_grid},
                        scoring="neg_mean_absolute_error",
                        cv=5, n_jobs=-1)
    grid.fit(X, y)
    return grid.best_estimator_.alpha, -grid.best_score_

# ---------- Boucle pseudo-OOS ----------
rows = []
models = []
preprocs = []
train_ends = []
cv_mae_history = []
alpha_history = []
last_fit_end = None
last_t_end = y_all.index.max() - relativedelta(months=h)

for t_end in y_all.index:
    if t_end > last_t_end:
        break
    y_tr = y_all.loc[:t_end]
    X_tr = X_all.loc[:t_end]
    if len(y_tr) < min_train_n:
        continue

    # Préproc appris sur TRAIN courant
    X_tr_p, prep = fit_preproc(X_tr, wins=winsor_level, do_norm=norm_var)

    # Tuning alpha sur TRAIN
    alpha, cv_mae = tune_ridge(X_tr_p, y_tr)
    cv_mae_history.append(cv_mae)
    alpha_history.append(alpha)

    # Fit final sur TRAIN
    model = Ridge(alpha=alpha, fit_intercept=True)
    model.fit(X_tr_p, y_tr.values)
    models.append(model)
    preprocs.append(prep)
    train_ends.append(t_end)
    last_fit_end = t_end

    # Prévision h=12
    t_fore = t_end + relativedelta(months=h)
    if t_fore in y_all.index:
        x_fore_raw = X_all.loc[[t_fore]]
        if use_bagging:
            yhat, dist, yhat_base = bagged_predict_ridge(
                X_tr_raw=X_tr, y_tr=y_tr, x_fore_raw=x_fore_raw,
                prep=prep, alpha=alpha, B=B_boot, L=L_block, rng=rng
            )
            y_p05 = float(np.percentile(dist, 5))
            y_p95 = float(np.percentile(dist, 95))
        else:
            yhat = float(model.predict(apply_preproc(x_fore_raw, prep))[0])
            yhat_base = yhat
            y_p05, y_p95 = (np.nan, np.nan)
        rows.append((t_fore, yhat, float(y_all.loc[t_fore]), yhat_base, y_p05, y_p95))

# ---------- DataFrame OOS ----------
if rows:
    df_oos = (
        pd.DataFrame(rows, columns=["date","y_pred","y_true","y_pred_base","y_pred_p05","y_pred_p95"])
        .assign(date=lambda d: pd.to_datetime(d["date"]).dt.to_period("M").dt.to_timestamp(how="start"))
        .set_index("date").sort_index()
    )
else:
    df_oos = pd.DataFrame(columns=["y_pred","y_true","y_pred_base","y_pred_p05","y_pred_p95"])
    df_oos.index = pd.to_datetime(pd.Index([]))

print(f"\n✅ Pseudo-OOS Ridge terminé — n prévisions = {len(df_oos)}")
print(df_oos.head(3))

# ---------- Scores ----------
def _scores(df):
    if len(df) == 0:
        return {"MAE": np.nan, "RMSE": np.nan, "R2": np.nan}
    mae = mean_absolute_error(df["y_true"], df["y_pred"])
    rmse = np.sqrt(mean_squared_error(df["y_true"], df["y_pred"]))
    r2 = r2_score(df["y_true"], df["y_pred"]) if len(df) > 1 else np.nan
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

df_val  = df_oos.loc[eval_start:eval_end].copy()
df_test = df_oos.loc[test_start:test_end].copy()
sc_val  = _scores(df_val)
sc_test = _scores(df_test)

print(f"\n📊 Validation 83–89 | n={len(df_val)} | MAE={sc_val['MAE']:.3f} | RMSE={sc_val['RMSE']:.3f} | R²={sc_val['R2']:.3f}")
print(f"📊 Test 90–2025     | n={len(df_test)} | MAE={sc_test['MAE']:.3f} | RMSE={sc_test['RMSE']:.3f} | R²={sc_test['R2']:.3f}")

# ---------- Sauvegardes (Ridge) ----------
bundle = {
    "oos_predictions": df_oos.reset_index(),
    "params": {
        "model": "Ridge + Bagging",
        "horizon": h,
        "min_train_n": min_train_n,
        "winsor_level": winsor_level,
        "norm_var": norm_var,
        "features": features,
        "eval_window": (str(eval_start.date()), str(eval_end.date())),
        "test_window": (str(test_start.date()), str(test_end.date())),
        # plan refit/retune
        "refit_every_months": refit_every_months,
        "retune_every_months": retune_every_months,
        # search (spécifique Ridge)
        "hyper_search": "GridSearchCV (alpha grid) + hv-block CV (5 folds, gap=12), scoring=MAE",
        "alpha_grid": list(alpha_grid) if "alpha_grid" in globals() else None,
        "best_alpha_last": (alpha_history[-1] if len(alpha_history) else None),
        # bagging
        "use_bagging": bool(use_bagging),
        "n_boot": n_boot,
        "bootstrap_proportion": bootstrap_proportion,
    },
    "meta": {
        "trained_until": str(last_fit_end.date()) if last_fit_end is not None else None,
        "index_freq": "MS",
        "n_obs_all": int(len(df_all)),
        "n_forecasts": int(len(df_oos)),
        "cv_mae_history": cv_mae_history,     # MAE CV à chaque retune
        "alpha_history": alpha_history,       # alpha utilisé à chaque refit
    },
    "train_fit_dates": pd.to_datetime(pd.Index(train_ends)) if len(train_ends) else pd.to_datetime(pd.Index([])),

    # ✅ Pour permutation_importance_pseudo_oos & SHAP
    "models":   models,     # liste des modèles Ridge "base" (un par refit)
    "preprocs": preprocs,   # liste des préproc (dict) alignés
}

with open(RIDGE_BUNDLE, "wb") as f:
    pickle.dump(bundle, f)

pd.DataFrame([{
    "model": "Ridge+Bagging",
    "horizon": h,
    "min_train_n": min_train_n,
    "winsor_level": winsor_level,
    "norm_var": norm_var,
    "refit_every_months": refit_every_months,
    "retune_every_months": retune_every_months,
    "use_bagging": bool(use_bagging),
    "n_boot": n_boot,
    "bootstrap_proportion": bootstrap_proportion,
    "trained_until": bundle["meta"]["trained_until"],
    "n_forecasts": bundle["meta"]["n_forecasts"],
    "best_alpha_last": bundle["params"]["best_alpha_last"]
}]).to_csv(RIDGE_META, index=False)

# Artefact dernier ensemble (optionnel : pour audit)
ridge_artifact = {
    "trained_until": bundle["meta"]["trained_until"],
    "horizon": h,
    "features": features,
    "n_models_base": len(models),
    "best_alpha_last": bundle["params"]["best_alpha_last"],
}
with open(RIDGE_LAST_PKL, "wb") as f:
    pickle.dump(ridge_artifact, f)

pd.DataFrame([{
    "trained_until": ridge_artifact["trained_until"],
    "n_features": len(features),
    "horizon": h,
    "n_models_base": ridge_artifact["n_models_base"],
    "best_alpha_last": ridge_artifact["best_alpha_last"],
}]).to_csv(RIDGE_LAST_META, index=False)

print(f"\n💾 Bundle OOS sauvegardé → {RIDGE_BUNDLE}")
print(f"💾 Méta bundle       → {RIDGE_META}")
print(f"💾 Dernier modèle    → {RIDGE_LAST_PKL}")
print(f"💾 Méta dernier fit  → {RIDGE_LAST_META}")

✅ Données prêtes : 1960-01-01 → 2025-08-01 | n=788 | freq=MS
Features (10): ['TB3MS', 'RPI', 'INDPRO', 'DPCERA3M086SBEA', 'S&P 500', 'BUSLOANS'] ...

✅ Pseudo-OOS Ridge terminé — n prévisions = 741
              y_pred  y_true  y_pred_base  y_pred_p05  y_pred_p95
date                                                             
1963-12-01  0.093785     0.0    -0.181263   -0.455216    1.220200
1964-01-01  0.348080    -0.1    -0.071449   -0.302035    1.165286
1964-02-01  0.390935    -0.5     0.058525   -0.508924    1.117637

📊 Validation 83–89 | n=84 | MAE=0.819 | RMSE=1.025 | R²=-0.346
📊 Test 90–2025     | n=428 | MAE=0.821 | RMSE=1.464 | R²=0.080

💾 Bundle OOS sauvegardé → ridge_regression.pkl
💾 Méta bundle       → ridge_regression_meta.csv
💾 Dernier modèle    → ridge_last_model.pkl
💾 Méta dernier fit  → ridge_last_model_meta.csv


# LightGBM

In [17]:
# ==========================================================
# LightGBM + Bagging (pseudo-OOS, h=12) — structure "comme Ridge"
# - Refit annuel, retune hyperparams tous les 36 mois (dès 1983)
# - hv-block CV (5 folds, gap=12), scoring=MAE
# - Winsorisation 1%/99% + normalisation (apprises sur TRAIN)
# - Bagging : n_boot bootstrap (proportions/indices), moyenne des prédictions
# - Bundle complet : oos_predictions + models + preprocs + train_fit_dates
# ==========================================================
import numpy as np
import pandas as pd
import pickle
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from lightgbm import LGBMRegressor

# ---------- Paramètres généraux ----------
h = 12
min_train_n = 36
winsor_level = 0.01
norm_var = True
target_col = "UNRATE"

# Fenêtres d’éval / test
eval_start = pd.Timestamp("1983-01-01")
eval_end   = pd.Timestamp("1989-12-31")
test_start = pd.Timestamp("1990-01-01")
test_end   = pd.Timestamp("2025-12-31")

# ---------- Refit/Retune ----------
refit_every_months  = 12     # refit chaque 12 mois
retune_every_months = 36     # retune hyperparams chaque 36 mois

# ---------- Bagging ----------
use_bagging = True
n_boot = 30
bootstrap_proportion = 1.0
rng = np.random.default_rng(12345)

# ---------- Fichiers de sortie ----------
LGBM_BUNDLE     = "lightgbm_regression.pkl"
LGBM_META       = "lightgbm_regression_meta.csv"
LGBM_LAST_PKL   = "LGBM_last_trained_model.pkl"
LGBM_LAST_META  = "LGBM_last_trained_model_meta.csv"

In [25]:
# ---------- Préparation df_stationary ----------
def _ensure_ms_index(df: pd.DataFrame) -> pd.DataFrame:
    """Index mensuel (MS). Si 'date' existe, on l'utilise comme index."""
    df = df.copy()
    if "date" in df.columns:
        df = df.set_index("date")
    idx = pd.to_datetime(df.index)
    df.index = idx.to_period("M").to_timestamp(how="start")
    return df.asfreq("MS")

# ⚠️ On suppose df_stationary dispo
df_all = _ensure_ms_index(df_stationary).sort_index()

if target_col not in df_all.columns:
    raise ValueError(f"La cible '{target_col}' est absente de df_stationary.")

y_all = df_all[target_col].astype(float)
X_all = df_all.drop(columns=[target_col]).astype(float)

# LightGBM : éviter espaces dans noms de colonnes
X_all.columns = [str(c).replace(" ", "_") for c in X_all.columns]
features = list(X_all.columns)

print(f"✅ Données prêtes : {df_all.index.min().date()} → {df_all.index.max().date()} | n={len(df_all)} | freq=MS")
print(f"Features ({len(features)}): {features[:6]}{' ...' if len(features)>6 else ''}")

# ---------- Préproc ----------
def fit_preproc(X: pd.DataFrame, wins=0.01, do_norm=True):
    lower = X.quantile(wins)
    upper = X.quantile(1 - wins)
    Xw = X.clip(lower=lower, upper=upper, axis=1)
    if do_norm:
        mean = Xw.mean()
        std  = Xw.std().replace(0, 1)
        Xn   = (Xw - mean) / std
        prep = {"lower": lower, "upper": upper, "mean": mean, "std": std, "norm": True}
        return Xn, prep
    else:
        prep = {"lower": lower, "upper": upper, "mean": None, "std": None, "norm": False}
        return Xw, prep

def apply_preproc(X: pd.DataFrame, prep: dict):
    Xp = X.clip(lower=prep["lower"], upper=prep["upper"], axis=1)
    if prep["norm"]:
        Xp = (Xp - prep["mean"]) / prep["std"].replace(0, 1)
    return Xp

✅ Données prêtes : 1960-01-01 → 2025-08-01 | n=788 | freq=MS
Features (10): ['TB3MS', 'RPI', 'INDPRO', 'DPCERA3M086SBEA', 'S&P_500', 'BUSLOANS'] ...


In [26]:
# ---------- hv-block CV ----------
class HVBlockCV:
    def __init__(self, n_splits=5, gap=12):
        self.n_splits = n_splits
        self.gap = gap
    def split(self, X, y=None, groups=None):
        n = len(X)
        fold_sizes = np.full(self.n_splits, n // self.n_splits, dtype=int)
        fold_sizes[: n % self.n_splits] += 1
        idx = np.arange(n)
        cur = 0
        for fs in fold_sizes:
            start, stop = cur, cur + fs
            test_idx = idx[start:stop]
            train_mask = np.ones(n, dtype=bool)
            left = max(0, start - self.gap)
            right = min(n, stop + self.gap)
            train_mask[left:right] = False
            train_idx = idx[train_mask]
            cur = stop
            if len(train_idx) == 0 or len(test_idx) == 0:
                continue
            yield train_idx, test_idx
    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

In [27]:
# ---------- Grille LightGBM (stabilité) ----------
param_dist = {
    "subsample":        [0.05,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.0],
    "colsample_bytree": [.2,.3,.4,.5,.6,.7,1.0],
    "num_leaves":       [2,3,4,5,8,10,20,40,70,100],
    "n_estimators":     [5,10,20,30,40,50,75,100],
    "max_depth":        [1,2,3,5,8,15,-1],          # -1 = illimité
    "reg_alpha":        [0, .1, 1, 2, 7, 10, 50, 100],
    "reg_lambda":       [0, .1, 1, 10, 20, 50, 100],
    "min_child_samples":[5,10,15],
    "min_split_gain":   [0.0, 0.01, 0.05],
}

In [28]:
def tune_lgbm(X_tr_p: pd.DataFrame, y_tr: np.ndarray, seed=12345):
    cv = HVBlockCV(n_splits=5, gap=12)
    base = LGBMRegressor(
        boosting_type="gbdt",
        objective="regression",
        importance_type="gain",
        random_state=seed,
        n_jobs=1,    # éviter sur-parallélisation quand RandomizedSearchCV utilise n_jobs=-1
        verbose=-1
    )
    rs = RandomizedSearchCV(
        estimator=base,
        param_distributions=param_dist,
        n_iter=100,
        scoring="neg_mean_absolute_error",
        cv=cv,
        n_jobs=-1,
        random_state=seed,
        refit=True,
        verbose=0
    )
    rs.fit(X_tr_p, y_tr)
    return rs.best_params_, float(-rs.best_score_), rs.best_estimator_

In [29]:
# ---------- Bootstrap utils ----------
def bootstrap_indices(n, proportion=1.0, seed=None):
    m = int(round(n * proportion))
    rng_local = np.random.default_rng(seed)
    return rng_local.integers(0, n, size=m, endpoint=False)

def bagged_predict_lgbm(X_tr_raw, y_tr, x_fore_raw, prep, best_params, B, proportion, seed0):
    """
    Bagging LightGBM :
      - préproc fixe (appris sur TRAIN original)
      - bootstrap simple d'indices (proportion)
      - fit LGBM(**best_params) et prédiction
      - retourne la moyenne + distribution + prédiction base (fit sur tout TRAIN)
    """
    # Base (référence) : fit sur tout TRAIN prétraité
    X_tr_p = apply_preproc(X_tr_raw, prep)
    base = LGBMRegressor(
        boosting_type="gbdt",
        objective="regression",
        importance_type="gain",
        n_jobs=1,
        random_state=seed0,
        verbose=-1,
        **best_params
    )
    base.fit(X_tr_p, y_tr.values)
    yhat_base = float(base.predict(apply_preproc(x_fore_raw, prep))[0])

    preds = []
    n = len(X_tr_raw)
    for b in range(B):
        ix = bootstrap_indices(n, proportion=proportion, seed=seed0 + b)
        Xb = X_tr_raw.iloc[ix]
        yb = y_tr.iloc[ix]
        Xb_p, _ = fit_preproc(Xb, wins=winsor_level, do_norm=norm_var)  # préproc appris sur bootstrap
        m = LGBMRegressor(
            boosting_type="gbdt",
            objective="regression",
            importance_type="gain",
            n_jobs=1,
            random_state=seed0 + b,
            verbose=-1,
            **best_params
        )
        m.fit(Xb_p, yb.values)
        preds.append(float(m.predict(apply_preproc(x_fore_raw, _))[0]))  # appliquer le prep du bootstrap (_)

    preds = np.array(preds)
    return float(np.mean(preds)), preds, yhat_base

In [31]:
# ---------- Boucle pseudo-OOS ----------
rows = []                 # (date, y_pred, y_true, y_pred_base, p05, p95)
models = []               # modèles "base" LGBM par refit (pour permutation/SHAP)
preprocs = []             # préprocs alignés
train_ends = []           # dates de refit
cv_mae_history = []       # historique MAE CV lors des retunes
best_params_hist = []     # historique des params

last_fit_end = None
last_t_end   = y_all.index.max() - relativedelta(months=h)
last_refit_t = None
last_tune_t  = None

best_params = {}          # params courants (remplis au 1er retune)
base_model = None         # sécurité si use_bagging=False
boot_seed = 12345         # reseed à chaque refit
seed0 = 12345

for t_end in y_all.index:
    if t_end > last_t_end:
        break

    y_tr = y_all.loc[:t_end]
    X_tr = X_all.loc[:t_end]
    if len(y_tr) < min_train_n:
        continue

    # cadence refit
    if last_refit_t is None:
        need_refit = True
    else:
        months_since_refit = (t_end.year - last_refit_t.year)*12 + (t_end.month - last_refit_t.month)
        need_refit = months_since_refit >= refit_every_months

    # cadence retune
    need_tune = False
    if t_end >= eval_start:
        if last_tune_t is None:
            need_tune = True
        else:
            months_since_tune = (t_end.year - last_tune_t.year)*12 + (t_end.month - last_tune_t.month)
            need_tune = months_since_tune >= retune_every_months

    # Préproc global (pour tuning/refit)
    X_tr_p_global, prep_global = fit_preproc(X_tr, wins=winsor_level, do_norm=norm_var)

    if need_tune:
        best_params, best_cv_mae, _ = tune_lgbm(X_tr_p_global, y_tr.values, seed=seed0)
        last_tune_t = t_end
        cv_mae_history.append(best_cv_mae)
        best_params_hist.append(best_params.copy())
    else:
        cv_mae_history.append(np.nan)
        best_params_hist.append(best_params.copy() if best_params else {})

    # Valeurs sûres si aucun retune n'a encore eu lieu
    if not best_params:
        best_params = dict(
            subsample=0.7, colsample_bytree=0.7, num_leaves=31,
            n_estimators=100, max_depth=-1, reg_alpha=0.0, reg_lambda=0.0,
            min_child_samples=10, min_split_gain=0.0
        )

    if need_refit:
        # Modèle "base" stocké pour permutation/SHAP
        base_model = LGBMRegressor(
            boosting_type="gbdt",
            objective="regression",
            importance_type="gain",
            n_jobs=1,
            random_state=seed0,
            verbose=-1,
            **best_params
        )
        base_model.fit(X_tr_p_global, y_tr.values)
        models.append(base_model)
        preprocs.append(prep_global)

        train_ends.append(t_end)
        last_refit_t = t_end
        last_fit_end = t_end

        # reseed bagging à chaque refit
        boot_seed += 9973

    # Prévision h=12
    t_fore = t_end + relativedelta(months=h)
    if t_fore in y_all.index:
        x_fore_raw = X_all.loc[[t_fore]]

        if use_bagging:
            yhat, dist, yhat_base = bagged_predict_lgbm(
                X_tr_raw=X_tr, y_tr=y_tr, x_fore_raw=x_fore_raw,
                prep=prep_global, best_params=best_params,
                B=n_boot, proportion=bootstrap_proportion, seed0=boot_seed
            )
            y_p05 = float(np.percentile(dist, 5))
            y_p95 = float(np.percentile(dist, 95))
        else:
            # sans bagging : utiliser / garantir un base_model
            if base_model is None:
                base_model = LGBMRegressor(
                    boosting_type="gbdt",
                    objective="regression",
                    importance_type="gain",
                    n_jobs=1,
                    random_state=seed0,
                    verbose=-1,
                    **best_params
                ).fit(X_tr_p_global, y_tr.values)
            yhat = float(base_model.predict(apply_preproc(x_fore_raw, prep_global))[0])
            yhat_base = yhat
            y_p05, y_p95 = (np.nan, np.nan)

        rows.append((t_fore, yhat, float(y_all.loc[t_fore]), yhat_base, y_p05, y_p95))

# ---------- DataFrame OOS ----------
if rows:
    df_oos = pd.DataFrame(
        rows,
        columns=["date", "y_pred", "y_true", "y_pred_base", "y_pred_p05", "y_pred_p95"]
    )
    # ✅ Conversion date → début de mois (MS)
    df_oos["date"] = pd.to_datetime(df_oos["date"]).dt.to_period("M").dt.to_timestamp(how="start")
    df_oos = df_oos.set_index("date").sort_index()
else:
    df_oos = pd.DataFrame(columns=["y_pred", "y_true", "y_pred_base", "y_pred_p05", "y_pred_p95"])
    df_oos.index = pd.to_datetime(pd.Index([]))

# ---------- Scores ----------
def _scores(df):
    if len(df) == 0:
        return {"MAE": np.nan, "RMSE": np.nan, "R2": np.nan}
    mae  = mean_absolute_error(df["y_true"], df["y_pred"])
    rmse = np.sqrt(mean_squared_error(df["y_true"], df["y_pred"]))
    ssr  = np.sum((df["y_true"] - df["y_pred"])**2)
    sst  = np.sum((df["y_true"] - df["y_true"].mean())**2)
    r2   = 1 - ssr/sst if sst > 0 else np.nan
    return {"MAE": float(mae), "RMSE": float(rmse), "R2": float(r2)}

df_val  = df_oos.loc[eval_start:eval_end].copy()
df_test = df_oos.loc[test_start:test_end].copy()

sc_val  = _scores(df_val)
sc_test = _scores(df_test)

print(f"\n📊 Validation 83–89 — n={len(df_val)} | MAE={sc_val['MAE']:.3f} | RMSE={sc_val['RMSE']:.3f} | R²={sc_val['R2']:.3f}")
print(f"📊 Test 90–{test_end.year} — n={len(df_test)} | MAE={sc_test['MAE']:.3f} | RMSE={sc_test['RMSE']:.3f} | R²={sc_test['R2']:.3f}")


📊 Validation 83–89 — n=84 | MAE=0.788 | RMSE=1.040 | R²=-0.385
📊 Test 90–2025 — n=428 | MAE=0.777 | RMSE=1.379 | R²=0.183


In [32]:
# ---------- Sauvegardes ----------
bundle = {
    "oos_predictions": df_oos.reset_index(),
    "params": {
        "model": "LightGBM + Bagging",
        "horizon": h,
        "min_train_n": min_train_n,
        "winsor_level": winsor_level,
        "norm_var": norm_var,
        "features": features,
        "eval_window": (str(eval_start.date()), str(eval_end.date())),
        "test_window": (str(test_start.date()), str(test_end.date())),
        # plan refit/retune
        "refit_every_months": refit_every_months,
        "retune_every_months": retune_every_months,
        # search
        "hyper_search": "RandomizedSearchCV (100 iters) + hv-block CV (5 folds, gap=12), scoring=MAE",
        "best_params_last": best_params.copy(),
        # bagging
        "use_bagging": bool(use_bagging),
        "n_boot": int(n_boot),
        "bootstrap_proportion": float(bootstrap_proportion),
    },
    "meta": {
        "trained_until": str(last_fit_end.date()) if last_fit_end is not None else None,
        "index_freq": "MS",
        "n_obs_all": int(len(df_all)),
        "n_forecasts": int(len(df_oos)),
        "cv_mae_history": cv_mae_history,
        "best_params_history": best_params_hist
    },
    "train_fit_dates": pd.to_datetime(pd.Index(train_ends)),

    # ✅ Pour permutation_importance_pseudo_oos & SHAP
    "models":   models,     # liste des modèles "base" LGBM (un par refit)
    "preprocs": preprocs,   # liste des préproc (dict) alignés aux modèles
}

with open(LGBM_BUNDLE, "wb") as f:
    pickle.dump(bundle, f)

pd.DataFrame([{
    "model": "LightGBM+Bagging",
    "horizon": h,
    "min_train_n": min_train_n,
    "winsor_level": winsor_level,
    "norm_var": norm_var,
    "refit_every_months": refit_every_months,
    "retune_every_months": retune_every_months,
    "use_bagging": bool(use_bagging),
    "n_boot": int(n_boot),
    "bootstrap_proportion": float(bootstrap_proportion),
    "trained_until": bundle["meta"]["trained_until"],
    "n_forecasts": bundle["meta"]["n_forecasts"]
}]).to_csv(LGBM_META, index=False)

# Artefact dernier ensemble (optionnel : pour audit)
lgbm_artifact = {
    "trained_until": bundle["meta"]["trained_until"],
    "horizon": h,
    "features": features,
    "n_models_base": len(models),
    "best_params_last": best_params.copy(),
}
with open(LGBM_LAST_PKL, "wb") as f:
    pickle.dump(lgbm_artifact, f)
pd.DataFrame([{
    "trained_until": lgbm_artifact["trained_until"],
    "n_features": len(features),
    "horizon": h,
    "n_models_base": lgbm_artifact["n_models_base"]
}]).to_csv(LGBM_LAST_META, index=False)

print(f"\n💾 Bundle OOS sauvegardé → {LGBM_BUNDLE}")
print(f"💾 Méta bundle       → {LGBM_META}")
print(f"💾 Dernier modèle    → {LGBM_LAST_PKL}")
print(f"💾 Méta dernier fit  → {LGBM_LAST_META}")


💾 Bundle OOS sauvegardé → lightgbm_regression.pkl
💾 Méta bundle       → lightgbm_regression_meta.csv
💾 Dernier modèle    → LGBM_last_trained_model.pkl
💾 Méta dernier fit  → LGBM_last_trained_model_meta.csv
